In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

all_data = pd.read_csv('sales所在周次节假日_剔除换货.csv')
all_data.rename(columns={'店铺名称':'store','地理位置指数':'position','年份':'years','周次':'weeks of year','是否包含法定节假日':'Isholiday','季节':'season','大类':'item','销量(剔除换货）':'sales','均价（剔除换货）':'prices','促销（剔除换货）':'discount'},inplace=True)
all_data['weeks'] = all_data['weeks of year']

In [2]:
a = all_data
week_5 = a.loc[lambda a : a['years'] == 2015]
week_6 = a.loc[lambda a : a['years'] == 2016]
week_6['weeks'] += 53
a = pd.concat([week_5,week_6])

/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
from sklearn import preprocessing

#map店铺名称
name = a['store']
le = preprocessing.LabelEncoder()
le.fit(name)
list(name)
le.transform(name)
tmp = le.transform(name)
a['store'] = tmp

#map合并大类
category = a['item']
le.fit(category)
list(category)
le.transform(category)
tmp = le.transform(category)
a['item'] = tmp

#map季节
season = a['season']
le.fit(season)
list(season)
le.transform(season)
tmp = le.transform(season)
a['season'] = tmp


In [4]:
df = a
masked_series = (df['years']==2016) & (df['weeks'].isin([53,54,55]))
masked_series2 = (df['years']==2016) & (~(df['weeks'].isin([53,54,55])))
df.loc[(masked_series), 'train_or_test'] = 'test'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df['years']==2015,:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))


Train shape: (8119, 12)
Test shape: (340, 12)


In [5]:
df_n = df.drop(['train_or_test'],axis=1)


In [6]:
#df_n['sales'] = np.log1p(df.sales.values)
df_n['prices'] = np.log1p(df.sales.values)
df_n.sample(2)

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
2815,4,7,2016,40,1,2,6,25,3.258097,0.427997,93
3485,5,1,2016,41,1,2,0,3,1.386294,0.359055,94


In [13]:
df_n.describe()

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
count,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.00000,18163.000000,18163.000000,18163.000000,18163.000000
mean,14.547156,5.501569,2015.552992,27.652701,0.185432,1.597864,5.41992,7.880031,1.697586,0.364135,56.961295
std,8.695615,2.597241,0.497198,15.181708,0.388658,1.048877,2.71015,16.436671,0.880375,0.089089,30.663260
min,0.000000,1.000000,2015.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.693147,0.032120,1.000000
25%,7.000000,7.000000,2015.000000,14.000000,0.000000,1.000000,3.00000,2.000000,1.098612,0.305955,31.000000
50%,14.000000,7.000000,2016.000000,28.000000,0.000000,2.000000,6.00000,4.000000,1.609438,0.368088,59.000000
75%,22.000000,7.000000,2016.000000,40.000000,0.000000,2.000000,7.00000,9.000000,2.302585,0.420665,84.000000
max,29.000000,7.000000,2016.000000,53.000000,1.000000,3.000000,11.00000,912.000000,6.816736,0.902023,106.000000


In [7]:
''''
def one_hot_encoder(df_n, ohe_cols=['store','item','weeks of years','isholiday','season','item']):

#    One-Hot Encoder function
    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))
    df_n = pd.get_dummies(df_n, columns=ohe_cols)
    print('New df shape:{}'.format(df.shape))
    return df_n
'''

"'\ndef one_hot_encoder(df_n, ohe_cols=['store','item','weeks of years','isholiday','season','item']):\n\n#    One-Hot Encoder function\n    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))\n    df_n = pd.get_dummies(df_n, columns=ohe_cols)\n    print('New df shape:{}'.format(df.shape))\n    return df_n\n"

In [8]:
train = df_n[lambda df_n: df_n['years']==2015]
t1 = df_n[lambda df_n: df_n['weeks']==54]
t2 = df_n[lambda df_n: df_n['weeks']==55]
t3 = df_n[lambda df_n: df_n['weeks']==56]
test = pd.concat([t1,t2,t3])

In [9]:
#train = one_hot_encoder(train, ohe_cols=['store','item','weeks of year','Isholiday','season','item']) 

In [10]:
y_train = train['sales']
y_test = test['sales']
X_train = train.drop('sales', axis=1)
X_test = test.drop('sales', axis=1)

In [11]:
import lightgbm as lgb
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
print('The R^2 of prediction is:', r2_score(y_test, y_pred))


Starting training...
[1]	valid_0's l2: 134.571	valid_0's l1: 6.85487
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 101.484	valid_0's l1: 6.11775
[3]	valid_0's l2: 76.7358	valid_0's l1: 5.45671
[4]	valid_0's l2: 58.2244	valid_0's l1: 4.85992
[5]	valid_0's l2: 45.1395	valid_0's l1: 4.334
[6]	valid_0's l2: 35.4103	valid_0's l1: 3.8554
[7]	valid_0's l2: 28.5823	valid_0's l1: 3.46747
[8]	valid_0's l2: 24.0066	valid_0's l1: 3.16348
[9]	valid_0's l2: 21.257	valid_0's l1: 2.89137
[10]	valid_0's l2: 19.7649	valid_0's l1: 2.64608
[11]	valid_0's l2: 18.2449	valid_0's l1: 2.41282
[12]	valid_0's l2: 15.5326	valid_0's l1: 2.1739
[13]	valid_0's l2: 15.061	valid_0's l1: 1.98987
[14]	valid_0's l2: 13.112	valid_0's l1: 1.79277
[15]	valid_0's l2: 11.6107	valid_0's l1: 1.62391
[16]	valid_0's l2: 10.9825	valid_0's l1: 1.48061
[17]	valid_0's l2: 10.5013	valid_0's l1: 1.34999
[18]	valid_0's l2: 10.1648	valid_0's l1: 1.23249
[19]	valid_0's l2: 9.53454	valid_0's l1: 1.12641
[20